In [1]:
import os
import json
import numpy as np
import math
import pandas as pd

In [2]:
# Chemins utiles
dir_path = os.getcwd()
# Dossier data
data_dir_path = os.path.join(dir_path, '0 - data')
# Dossier fiches métiers scraping
output_dir_path = os.path.join(dir_path, '00 - Scraping output')
# Fichier Onisep links
onisep_link_file = os.path.join(data_dir_path, 'Onisep_metier_link.csv')
# Liste des métiers
jobs_list = os.listdir(output_dir_path)
len(jobs_list)

734

In [3]:
onisep_links = pd.read_csv(onisep_link_file, sep=';', header=0)
onisep_links.head()

,libellé métier,lien site onisep.fr
0,technicien/ne de fabrication de mobilier et de...,http://www.onisep.fr/http/redirection/metier/i...
1,agenceur/euse de cuisines et salles de bains,http://www.onisep.fr/http/redirection/metier/i...
2,chef/fe de fabrication des industries graphiques,http://www.onisep.fr/http/redirection/metier/i...
3,conducteur/trice de machines à imprimer,http://www.onisep.fr/http/redirection/metier/i...
4,opérateur/trice prépresse,http://www.onisep.fr/http/redirection/metier/i...


In [4]:
fiche_path = os.path.join(output_dir_path, '10002.json')
with open(fiche_path) as f:
    metier = json.load(f)
    f.close()

list(metier.keys())

['Métiers Associés ',
 'Synonymes ',
 'Secteur(s) professionnel(s) ',
 "Où l'exercer",
 'Métier',
 'Salaire débutant  ',
 'chapo',
 "Niveau d'accès ",
 "Centre(s) d'intérêt ",
 'Le métier',
 'Carrière et salaire',
 'Statut(s) ']

In [5]:
interet_liste = []
secteur_liste = []
niveau_liste = []
salaire_liste = []
statut_liste = []
for identifiant in jobs_list:
    fiche_path = os.path.join(output_dir_path, identifiant)
    with open(fiche_path) as f:
        metier = json.load(f)
        f.close()
    #print(metier)
    try:
        for interet in metier["Centre(s) d'intérêt "]:
            if interet not in interet_liste:
                interet_liste.append(interet)
    except:
        pass
    
    try:
        for secteur in metier["Secteur(s) professionnel(s) "]:
            if secteur not in secteur_liste:
                secteur_liste.append(secteur)
    except:
        pass

    try:
        for niveau in metier["Niveau d'accès "]:
            if niveau not in niveau_liste:
                niveau_liste.append(niveau)
    except:
        pass
    
    try:
        for salaire in metier["Salaire débutant  "]:
            if salaire not in salaire_liste:
                salaire_liste.append(salaire)
    except:
        pass
    
    try:
        for statut in metier["Statut(s) "]:
            if statut not in statut_liste:
                statut_liste.append(statut)
    except:
        pass

In [31]:
print("Nombre de centre d'intérêt =", len(interet_liste))
print("Nombre de secteurs professionnels =", len(secteur_liste))
print("Nombre de niveaux d'accès =", len(niveau_liste))
print("Nombre de salaire débutants =", len(salaire_liste))
print("Nombre de statut =", len(statut_liste))
niveau_liste

Nombre de centre d'intérêt = 25
Nombre de secteurs professionnels = 32
Nombre de niveaux d'accès = 14
Nombre de salaire débutants = 133
Nombre de statut = 9


['sans diplôme',
 'bac + 2',
 'bac ou équivalent',
 'bac + 5',
 'CAP ou équivalent',
 'bac + 3',
 'bac + 4',
 'bac + 9 et plus',
 'bac + 8',
 'diplôme national du brevet',
 '3e',
 'bac + 6',
 'bac + 7',
 'bac + 1']

In [30]:

df = pd.DataFrame(columns = ['Code Onisep', 'Métier'] + interet_liste + secteur_liste + statut_liste
                 + ['Niveau diplome', 'Salaire débutant'])

j = 0
for job in jobs_list:
    code_oni = job[0:6]
    if code_oni[5]=='.':
        code_oni=code_oni[0:5]
    df.loc[j,'Code Onisep'] = code_oni
    
    fiche_path = os.path.join(output_dir_path, job)
    with open(fiche_path) as f:
        metier = json.load(f)
        f.close()      
    #print(metier)
    df.loc[j,'Métier'] = metier["Métier"] 
    try:
        for interet in interet_liste:
            if interet in metier["Centre(s) d'intérêt "]:
                df.loc[j,str(interet)] = 1
        
            else:
                df.loc[j,str(interet)] = 0
    except:
        for interet in interet_liste:
            df.loc[j,str(interet)] = 0
    
    try:
        for secteur in secteur_liste:
            if secteur in metier["Secteur(s) professionnel(s) "]:
                df.loc[j,secteur] = 1
        
            else:
                df.loc[j,secteur] = 0
    except:
        for interet in interet_liste:
            df.loc[j,secteur] = 0
            
    try:
        for statut in statut_liste:
            if statut in metier["Statut(s) "]:
                df.loc[j,statut] = 1
        
            else:
                df.loc[j,statut] = 0
    except:
        for statut in statut_liste:
            df.loc[j,statut] = 0
    try:    
        if metier["Niveau d'accès "][0]=='3e':
            df.loc[j, 'Niveau diplome'] = -2
        elif metier["Niveau d'accès "][0]=='sans diplôme':
            df.loc[j, 'Niveau diplome'] = -3
        elif metier["Niveau d'accès "][0]=='diplôme national du brevet':
            df.loc[j, 'Niveau diplome'] = -1
        elif metier["Niveau d'accès "][0]=='CAP ou équivalent':
            df.loc[j, 'Niveau diplome'] = 0 
        elif metier["Niveau d'accès "][0]=='bac ou équivalent':
            df.loc[j, 'Niveau diplome'] = 1    
        elif metier["Niveau d'accès "][0]=='bac + 1':
            df.loc[j, 'Niveau diplome'] = 2
        elif metier["Niveau d'accès "][0]=='bac + 2':
            df.loc[j, 'Niveau diplome'] = 3
        elif metier["Niveau d'accès "][0]=='bac + 3':
            df.loc[j, 'Niveau diplome'] = 4    
        elif metier["Niveau d'accès "][0]=='bac + 4':
            df.loc[j, 'Niveau diplome'] = 5  
        elif metier["Niveau d'accès "][0]=='bac + 5':
            df.loc[j, 'Niveau diplome'] = 6    
        elif metier["Niveau d'accès "][0]=='bac + 6':
            df.loc[j, 'Niveau diplome'] = 7    
        elif metier["Niveau d'accès "][0]=='bac + 7':
            df.loc[j, 'Niveau diplome'] = 8    
        elif metier["Niveau d'accès "][0]=='bac + 8':
            df.loc[j, 'Niveau diplome'] = 9    
        elif metier["Niveau d'accès "][0]=='bac + 9 et plus':
            df.loc[j, 'Niveau diplome'] = 10
    except:
        df.loc[j, 'Niveau diplome'] = 0

    try:
        if metier["Salaire débutant  "][0]=='Smic. euros' or metier["Salaire débutant  "][0]=='Smic euros':
            df.loc[j,'Salaire débutant'] = 1500
        elif metier["Salaire débutant  "][0]=='1718euros brut par mois (dans le public) et 1829 euros brut par mois (dans le privé). euros':
            df.loc[j,'Salaire débutant'] = 1773
        elif metier["Salaire débutant  "][0]=='Variable euros':
            df.loc[j,'Salaire débutant'] = np.nan
        elif metier["Salaire débutant  "][0]=='De1528 à 1962 euros':
            df.loc[j,'Salaire débutant'] = 1745
        else:
            salaire = metier["Salaire débutant  "][0][0:4]
            if salaire[3]!=' ':
                df.loc[j,'Salaire débutant'] = salaire
            else:
                df.loc[j,'Salaire débutant'] = salaire[0:3]
    except:
        df.loc[j,'Salaire débutant'] = np.nan
        
    j += 1
#for interet in interet_liste:
#df[['Code Onisep','Métier']] = onisep_links
df.head()

,Code Onisep,Métier,Le sport est ma passion,Je suis branché high tech,Je veux travailler de mes mains,Réparer - bricoler - j'adore ça,J'ai le sens du contact,Je veux protéger la planète,J'aime bouger,J'aime les sensations fortes,...,Statut artisan,Statut fonctionnaire,Indépendant,Militaire,Libéral,Saisonnier,Contrat de volontariat,Intermittent du spectacle,Niveau diplome,Salaire débutant
0,10293,sportif / sportive de haut niveau,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-3,NaN
1,100365,testeur / testeuse,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1700
2,10358,mécanicien-outilleur / mécanicienne-outilleuse,0,1,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1500
3,100647,consultant / consultante en conduite de change...,0,0,0,0,1,0,0,0,...,0,1,1,0,0,0,0,0,6,2916
4,10262,chargé / chargée hygiène sécurité environnemen...,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,3,1466
5,10274,officier de marine / officière de marine,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,3,1648
6,100313,garde à cheval,0,0,0,0,0,1,1,0,...,0,1,0,1,0,0,0,0,0,NaN
7,10201,éditeur / éditrice,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,6,1890
8,10231,psychomotricien / psychomotricienne,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,4,1466
9,10184,concepteur-rédacteur / conceptrice-rédactrice,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,3,2130


In [23]:
fiche_path

'/home/timothee/PycharmProjects/ImpalaPoleEmploi2/Impala/Onisep/tsne/00 - Scraping output/10184.json'

In [32]:
output_path = os.path.join(dir_path, 'table_onisep.csv')
df.to_csv(output_path)
    